# o'reillyのツノガレイ強化学習の本

## 深層学習で方策勾配法

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
- [方策勾配法](#方策勾配法)
  - [エージェントの実装1](#エージェントの実装1)
  - [エージェントの実行1](#エージェントの実行1)
- [REINFORCE](#REINFORCE)
  - [エージェントの実装2](#エージェントの実装2)
  - [エージェントの実行2](#エージェントの実行2)
- [Actor-Critic](#Actor-Critic)
  - [エージェントの実装3](#エージェントの実装3)
  - [エージェントの実行3](#エージェントの実行3)

## 参考
- https://github.com/oreilly-japan/deep-learning-from-scratch-4/tree/master/ch01
- [強化学習（Reinforcement Learning） - .NET 開発基盤部会 Wiki](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92%EF%BC%88Reinforcement%20Learning%EF%BC%89)

## 環境準備

### インストール

In [ ]:
!pip install numpy
!pip install tabulate
!pip install matplotlib
!pip install dezero
!pip install dezerogym
!pip install gym[classic_control]

### インポート

In [ ]:
import numpy as np
import copy
import random
from collections import deque
from tabulate import tabulate
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# 廃止となったエイリアスを書く
np.object = object
np.bool = bool
np.int = int
np.float = float
np.typeDict = {k: v for k, v in np.sctypeDict.items() if isinstance(v, type)}

from dezero import Variable
from dezero import Model
from dezero import optimizers
import dezero.functions as F
import dezero.layers as L

from dezerogym.gridworld import GridWorld
import gym # OpenAI Gym

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

### 共通関数

#### 総報酬表示

In [ ]:
def plot_total_reward(reward_history):
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.plot(range(len(reward_history)), reward_history)
    plt.show()

#### Policy

In [ ]:
class Policy(Model):
    def __init__(self, action_size):
        super().__init__()
        self.l1 = L.Linear(128)
        self.l2 = L.Linear(action_size)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.softmax(self.l2(x))
        return x

#### PolicyNetとValueNet

In [ ]:
class PolicyNet(Model):
    def __init__(self, action_size=2):
        super().__init__()
        self.l1 = L.Linear(128)
        self.l2 = L.Linear(action_size)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        x = F.softmax(x)
        return x

class ValueNet(Model):
    def __init__(self):
        super().__init__()
        self.l1 = L.Linear(128)
        self.l2 = L.Linear(1)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x

### 方策勾配法
- 方策勾配法は、エージェントの行動方策を直接最適化する手法であり、連続的な状態空間や行動空間に適しています。
- 深層学習を用いることで、複雑な方策を学習できるため、現実的な応用において重要な手法となっている。
- 代表的な方策勾配法にはREINFORCEやActor-Critic、PPOなどがある。

#### エージェントの実装1

In [ ]:
class Agent1:
    def __init__(self):
        self.gamma = 0.98
        self.lr = 0.0002
        self.action_size = 2

        # 行動履歴
        self.memory = []
        
        # 方策NN
        self.pi = Policy(self.action_size)
        self.optimizer = optimizers.Adam(self.lr)
        self.optimizer.setup(self.pi)

    # 方策からアクションを決める。
    def get_action(self, state):
        state = state[np.newaxis, :] # add batch axis
        probs = self.pi(state)
        probs = probs[0]
        action = np.random.choice(len(probs), p=probs.data)
        return action, probs[action]

    # 行動履歴の記録
    def add(self, reward, prob):
        data = (reward, prob)
        self.memory.append(data)

    # エピソードを学習（≒ミニバッチ学習）
    def update(self):
        
        # 初期化
        G, loss = 0, 0
        self.pi.cleargrads()
        
        # 行動履歴から逆再生して報酬を計算
        for reward, prob in reversed(self.memory):
            G = reward + self.gamma * G

        # 損失関数で損失計算
        # 収益を行動確率を重みに合計
        for reward, prob in self.memory:
            loss += -F.log(prob) * G

        # 微分（誤差逆伝播）
        loss.backward() # loss = 損失関数()の微分
        self.optimizer.update() # ミニバッチ学習で更新
        self.memory = [] # 履歴のクリア

#### エージェントの実行1

In [ ]:
env = gym.make('CartPole-v1')
agent = Agent1()

# 推移の確認用のリストを初期化
reward_history = []

# 3000エピソード
episodes = 3000
for episode in range(episodes):
    
    # 環境の初期化
    state, info = env.reset()
    # その他の初期化
    total_reward = 0
    done = False

    # done=tureまでが1エピソード
    while not done:
        
        # ActionでStep
        action, prob = agent.get_action(state)
        next_state, reward, done, truncated, info = env.step(action)

        # エピソードを記録
        agent.add(reward, prob)
        
        state = next_state
        total_reward += reward

    # エピソードを学習（≒ミニバッチ学習）
    agent.update()

    # 結果を記録
    reward_history.append(total_reward)
    if episode % 100 == 0:
        print("episode :{}, total reward : {:.1f}".format(episode, total_reward))

# plot
plot_total_reward(reward_history)

### REINFORCE
- 最も基本的な方策勾配アルゴリズム。
- エピソードごとに累積報酬を用いて方策の更新を行う。

#### エージェントの実装2

In [ ]:
class Agent2:
    def __init__(self):
        self.gamma = 0.98
        self.lr = 0.0002
        self.action_size = 2

        # 行動履歴
        self.memory = []
        
        # 方策NN
        self.pi = Policy(self.action_size)
        self.optimizer = optimizers.Adam(self.lr)
        self.optimizer.setup(self.pi)

    # 方策からアクションを決める。
    def get_action(self, state):
        state = state[np.newaxis, :] # add batch axis
        probs = self.pi(state)
        probs = probs[0]
        action = np.random.choice(len(probs), p=probs.data)
        return action, probs[action]

    # 行動履歴の記録
    def add(self, reward, prob):
        data = (reward, prob)
        self.memory.append(data)

    # エピソードを学習（≒ミニバッチ学習）
    def update(self):
        
        # 初期化
        G, loss = 0, 0
        self.pi.cleargrads()

        # 行動履歴から逆再生して報酬を計算
        for reward, prob in reversed(self.memory):
            G = reward + self.gamma * G
            
            # 損失関数で損失計算
            # 収益を行動確率を重みに合計
            # 方策勾配法と違って計算途中の収益Gを使っている。
            loss += -F.log(prob) * G

        # 微分（誤差逆伝播）
        loss.backward() # loss = 損失関数()の微分
        self.optimizer.update() # ミニバッチ学習で更新
        self.memory = [] # 履歴のクリア

#### エージェントの実行2

In [ ]:
env = gym.make('CartPole-v1')
agent = Agent2()

# 推移の確認用のリストを初期化
reward_history = []

# 3000エピソード
episodes = 3000
for episode in range(episodes):
    
    # 環境の初期化
    state, info = env.reset()
    # その他の初期化
    total_reward = 0
    done = False

    # done=tureまでが1エピソード
    while not done:
        
        # ActionでStep
        action, prob = agent.get_action(state)
        next_state, reward, done, truncated, info = env.step(action)

        # エピソードを記録
        agent.add(reward, prob)
        
        state = next_state
        total_reward += reward

    # エピソードを学習（ミニバッチ学習）
    agent.update()

    # 結果を記録
    reward_history.append(total_reward)
    if episode % 100 == 0:
        print("episode :{}, total reward : {:.1f}".format(episode, total_reward))

# plot
plot_total_reward(reward_history)

### Actor-Critic
- 方策を更新するアクター（Actor）と、行動価値関数を近似するクリティック（Critic）を組み合わせる手法。
- クリティックは、状態価値関数や行動価値関数を学習し、その評価に基づいてアクターが方策を更新する。

#### エージェントの実装3

In [ ]:
class Agent3:
    def __init__(self):
        self.gamma = 0.98
        self.lr_pi = 0.0002
        self.lr_v = 0.0005
        self.action_size = 2

        self.pi = PolicyNet() # Actor
        self.optimizer_pi = optimizers.Adam(self.lr_pi).setup(self.pi)
        self.v = ValueNet() # Critic
        self.optimizer_v = optimizers.Adam(self.lr_v).setup(self.v)

    # 方策（Actor）からアクションを決める。
    def get_action(self, state):
        state = state[np.newaxis, :] # add batch axis
        probs = self.pi(state)
        probs = probs[0]
        action = np.random.choice(len(probs), p=probs.data)
        return action, probs[action]

    # オンライン学習
    def update(self, state, action_prob, reward, next_state, done):
        state = state[np.newaxis, :] # add batch axis
        next_state = next_state[np.newaxis, :] # add batch axis

        # ========== (1) Update V network ===========
        v = self.v(state)
        
        # 以下の式はDQNのQ学習 
        # target = reward + (1 - done) * self.gamma * next_q_max
        target = reward + (1 - done) * self.gamma * self.v(next_state)
        
        # 損失関数で損失計算
        target.unchain() # 勾配の計算から除外
        loss_v = F.mean_squared_error(v, target)

        # ========== (2) Update pi network ===========
        delta = target - v
        
        # 損失関数で損失計算
        delta.unchain() # 勾配の計算から除外
        loss_pi = -F.log(action_prob) * delta

        # ============================================
        self.v.cleargrads()
        self.pi.cleargrads()
        
        # 微分（誤差逆伝播）
        loss_v.backward() # loss_v = 損失関数()の微分
        loss_pi.backward() # loss_pi = 損失関数()の微分
        
        # オンライン学習で更新
        self.optimizer_v.update()
        self.optimizer_pi.update()

#### エージェントの実行3

In [ ]:
env = gym.make('CartPole-v1')
agent = Agent3()

# 推移の確認用のリストを初期化
reward_history = []

# 3000エピソード
episodes = 3000
for episode in range(episodes):
    
    # 環境の初期化
    state, info = env.reset()
    # その他の初期化
    total_reward = 0
    done = False

    # done=tureまでが1エピソード
    while not done:
        
        # ActionでStep
        action, prob = agent.get_action(state)
        next_state, reward, done, truncated, info = env.step(action)
        
        # 都度学習（オンライン学習）
        agent.update(state, prob, reward, next_state, done)

        state = next_state
        total_reward += reward

    # 結果を記録
    reward_history.append(total_reward)
    if episode % 100 == 0:
        print("episode :{}, total reward : {:.1f}".format(episode, total_reward))

# plot
plot_total_reward(reward_history)